In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 2
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 71

In [1]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [2]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [3]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [4]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [5]:
%%sql
SELECT * FROM halloffame LIMIT 5

Run the following cell for grading purposes.

In [6]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [7]:
sample_query_cost = ...
sample_query_timing = ...

In [ ]:
grader.check("q0")

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [9]:
%%sql result_1a <<
...

In [10]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [ ]:
grader.check("q1a")

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [13]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

In [14]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

In [15]:
inducted_hof_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

In [16]:
provided_query_cost = ...
provided_query_timing = ...
your_query_cost = ...
your_query_timing = ...

In [ ]:
grader.check("q1bi")

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [19]:
q1b_part2 = ...

In [ ]:
grader.check("q1bii")

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [24]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
...
;
SELECT * FROM inducted_hof_ca;

In [25]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [ ]:
grader.check("q1c")

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [28]:
%%sql with_view <<
...

In [29]:
%%sql without_view <<
...

In [30]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [ ]:
grader.check("q1di")

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [36]:
with_view_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

In [37]:
without_view_query_str = ...
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

In [38]:
with_view_cost = ...
with_view_timing = ...
without_view_cost = ...
without_view_timing = ...

In [ ]:
grader.check("q1dii")

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [41]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
...
;
SELECT * FROM inducted_hof_ca_mat;

In [42]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [ ]:
grader.check("q1ei")

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [45]:
%%sql with_materialized_view <<
...

In [46]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [ ]:
grader.check("q1eii")

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [49]:
with_materialized_view_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

In [50]:
with_materialized_view_cost = ...
with_materialized_view_timing = ...

In [ ]:
grader.check("q1eiii")

#### Question 1eiv:
Given your findings from inspecting the query plans of queries using a view, no view, and a materialized view, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [53]:
q1e_part4 = ...

In [ ]:
grader.check("q1eiv")

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [58]:
query_view_str = ...
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

In [59]:
query_view_cost = ...
query_view_timing = ...

In [ ]:
grader.check("q2a")

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [61]:
%%sql result_2b <<
...

In [62]:
query_view_with_filter_str = ...
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

In [63]:
query_view_with_filter_cost = ...
query_view_with_filter_timing = ...

In [ ]:
grader.check("q2b")

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [66]:
q2c = ...

In [ ]:
grader.check("q2c")

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [69]:
%%sql q3a_join <<
...

Inspect the query plan for the query you wrote above.

In [70]:
q3a_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [71]:
q3a = ...

In [ ]:
grader.check("q3a")

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [74]:
%%sql q3b_join <<
...

Inspect the query plan for the query you wrote above.

In [75]:
q3b_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [76]:
q3b = ...

In [ ]:
grader.check("q3b")

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [79]:
%%sql q3c_join <<
...

Inspect the query plan for the query you wrote above.

In [80]:
q3c_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [81]:
q3c = ...

In [ ]:
grader.check("q3c")

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Inspect the query plan and record the execution time and cost.

In [84]:
%%sql result_4a <<
...

In [85]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [ ]:
grader.check("q4ai")

In [88]:
result_4a_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

In [89]:
result_4a_cost = ...
result_4a_timing = ...

In [ ]:
grader.check("q4aii")

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [92]:
%%sql result_4b <<
...

Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [93]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

In [94]:
result_4b_cost = ...
result_4b_timing = ...

In [ ]:
grader.check("q4b")

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [97]:
%%sql 
...

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [98]:
%%sql result_4c <<
...

Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [99]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

In [100]:
result_4c_cost = ...
result_4c_timing = ...

In [ ]:
grader.check("q4c")

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [103]:
q4d_part1 = ...

In [ ]:
grader.check("q4di")

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [108]:
%%sql 
...

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games. Your query should join the `salaries` and `appearances` table on `yearid`, `teamid`, and `playerid`.

In [109]:
%%sql result_4ei <<
...

In [110]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [ ]:
grader.check("q4ei_part1")

Inspect the query plan and record the execution time and cost.

In [113]:
result_4ei_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

In [114]:
result_4ei_cost = ...
result_4ei_timing = ...

In [ ]:
grader.check("q4ei_part2")

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [116]:
%%sql result_4eii << 
...

In [117]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

In [118]:
result_4eii_with_index_cost = ...
result_4eii_with_index_timing = ...

In [ ]:
grader.check("q4eii")

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [121]:
%%sql 
...

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on `yearid`, `teamid`, and `playerid`.

In [122]:
%%sql result_4eiii <<
...

In [123]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [ ]:
grader.check("q4eiii_part1")

In [126]:
result_4eiii_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

In [127]:
result_4eiii_cost = ...
result_4eiii_timing = ...

In [ ]:
grader.check("q4eiii_part2")

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [129]:
%%sql result_4eiv << 
...

In [130]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

In [131]:
result_4eiv_with_index_cost = ...
result_4eiv_with_index_timing = ...

In [ ]:
grader.check("q4eiv")

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [134]:
%%sql 
...

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [135]:
%%sql result_4v <<
...

In [136]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

In [137]:
result_4ev_multiple_col_index_cost = ...
result_4ev_multiple_col_index_timing = ...

In [ ]:
grader.check("q4ev")

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [140]:
q4e_part6 = ...

In [ ]:
grader.check("q4evi")

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [145]:
%%sql result_5a_min << 
...

In [146]:
result_5a_min_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

In [147]:
result_5a_min_query_cost = ...
result_5a_min_query_timing = ...

In [ ]:
grader.check("q5ai")

In [149]:
%%sql result_5a_avg <<
...

In [150]:
result_5a_avg_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

In [151]:
result_5a_avg_query_cost = ...
result_5a_avg_query_timing = ...

In [ ]:
grader.check("q5aii")

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [154]:
%%sql 
...

In [155]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

In [156]:
result_5b_min_query_cost = ...
result_5b_min_query_timing = ...

In [ ]:
grader.check("q5bi")

In [159]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

In [160]:
result_5b_avg_query_cost = ...
result_5b_avg_query_timing = ...

In [ ]:
grader.check("q5bii")

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [163]:
%%sql 
...

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [164]:
q5c = ...

In [ ]:
grader.check("q5c")

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [167]:
%%sql result_6a <<
...

In [168]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [ ]:
grader.check("q6ai")

In [171]:
result_6a_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [172]:
result_6a_cost = ...
result_6a_timing = ...

In [ ]:
grader.check("q6aii")

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [175]:
%%sql 
...

In [176]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [177]:
result_6b_cost = ...
result_6b_timing = ...

In [ ]:
grader.check("q6b")

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [180]:
%%sql 
...

In [181]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [182]:
result_6c_cost = ...
result_6c_timing = ...

In [ ]:
grader.check("q6c")

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [185]:
q6d = ...

In [ ]:
grader.check("q6d")

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [190]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

Next, run the provided update script and record the **wall time**. Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [191]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [192]:
result_7a_timing = ...

In [ ]:
grader.check("q7a")

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [194]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

### Question 7b:
Now, create an index on the `salary` column and record the wall time after executing the update script.

In [195]:
%%sql 
...

In [196]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100000..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [197]:
result_7b_timing = ...

In [ ]:
grader.check("q7b")

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q9
manual: true
points: 6
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [199]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()